# **Launch Sites Locations Analysis with Folium**


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [11]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
ls = folium.map.FeatureGroup()
for lat, lng in zip (launch_sites_df.Lat, launch_sites_df.Long):
    ls.add_child(
        folium.vector_layers.CircleMarker(
            [lat,lng],
             radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )
site_map.add_child(ls) 

In [12]:
spacex_df["Launch Site"].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [13]:
marker_cluster = MarkerCluster()


In [14]:

# Initialize an empty list to store marker colors
marker_colors = []

# Loop through each row in the dataframe
for index, row in spacex_df.iterrows():
    # Check the value of 'class' and assign the appropriate marker color
    if row['class'] == 1:
        marker_colors.append('green')
    else:
        marker_colors.append('red')

# Add the 'marker_color' list as a new column to the dataframe
spacex_df['marker_color'] = marker_colors

# Display the first few rows to verify
spacex_df.head()


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [15]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)


# Loop through each row in the spacex_df DataFrame
for index, record in spacex_df.iterrows():
    # Create a folium Marker with the appropriate icon color based on the 'marker_color' column
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],  # Coordinates of the launch site
        icon=folium.Icon(color='white', icon_color=record['marker_color']),  # Customize the marker icon color
        popup=f"Launch Site: {record['Launch Site']}\nSuccess: {record['class']}"  # Add a popup with details
    )
    
    # Add the marker to the marker cluster
    marker_cluster.add_child(marker)

# Display the map
site_map


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [24]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [18]:
import folium
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Function to calculate the distance between two coordinates (Haversine formula)
def calculate_distance(lat1, lon1, lat2, lon2):
    # Approximate radius of Earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Coordinates of the closest coastline
coastline_lat = 28.56367
coastline_lon = -80.57163

# Create a map object
site_map = folium.Map(location=[28.5, -80.6], zoom_start=10)

# Create a MarkerCluster to add multiple markers
marker_cluster = folium.plugins.MarkerCluster()

# Loop through each row in spacex_df DataFrame to add markers for launch sites
for index, record in spacex_df.iterrows():
    # Create a marker for each launch site
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],  # Coordinates of the launch site
        icon=folium.Icon(color='white', icon_color=record['marker_color']),  # Customize the icon
        popup=f"Launch Site: {record['Launch Site']}\nSuccess: {record['class']}"  # Popup information
    )
    marker_cluster.add_child(marker)  # Add the marker to the marker cluster

# Add the marker cluster to the map
site_map.add_child(marker_cluster)

# Calculate the distance between the launch site and the coastline
launch_lat = spacex_df.loc[0, 'Lat']  # Example launch site coordinates
launch_lon = spacex_df.loc[0, 'Long']
distance_coastline = calculate_distance(launch_lat, launch_lon, coastline_lat, coastline_lon)

# Create a marker at the coastline and display the distance to the launch site
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline)
    )
)
site_map.add_child(distance_marker)

# Create and add a PolyLine to represent the line between the launch site and the coastline
coordinates = [[launch_lat, launch_lon], [coastline_lat, coastline_lon]]
line = folium.PolyLine(locations=coordinates, weight=2, color='blue')
site_map.add_child(line)

# Display the map
site_map
# Example coordinates for the closest city, railway, and highway (replace with actual coordinates)
closest_city_lat = 28.3922  # Example: Orlando, FL
closest_city_lon = -81.2298

closest_railway_lat = 28.5721  # Example railway coordinates
closest_railway_lon = -80.5850

closest_highway_lat = 28.4550  # Example highway coordinates
closest_highway_lon = -80.7090

# Calculate distances to each of these locations from the launch site
distance_city = calculate_distance(launch_lat, launch_lon, closest_city_lat, closest_city_lon)
distance_railway = calculate_distance(launch_lat, launch_lon, closest_railway_lat, closest_railway_lon)
distance_highway = calculate_distance(launch_lat, launch_lon, closest_highway_lat, closest_highway_lon)

# Create a marker for the closest city and display the distance
city_marker = folium.Marker(
    location=[closest_city_lat, closest_city_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#2980b9;"><b>{:.2f} KM to City</b></div>'.format(distance_city)
    )
)
site_map.add_child(city_marker)

# Create a marker for the closest railway and display the distance
railway_marker = folium.Marker(
    location=[closest_railway_lat, closest_railway_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#e74c3c;"><b>{:.2f} KM to Railway</b></div>'.format(distance_railway)
    )
)
site_map.add_child(railway_marker)

# Create a marker for the closest highway and display the distance
highway_marker = folium.Marker(
    location=[closest_highway_lat, closest_highway_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#27ae60;"><b>{:.2f} KM to Highway</b></div>'.format(distance_highway)
    )
)
site_map.add_child(highway_marker)

# Draw lines between the launch site and the closest city, railway, and highway
city_line = folium.PolyLine(locations=[[launch_lat, launch_lon], [closest_city_lat, closest_city_lon]], color='blue', weight=2)
railway_line = folium.PolyLine(locations=[[launch_lat, launch_lon], [closest_railway_lat, closest_railway_lon]], color='red', weight=2)
highway_line = folium.PolyLine(locations=[[launch_lat, launch_lon], [closest_highway_lat, closest_highway_lon]], color='green', weight=2)

# Add the lines to the map
site_map.add_child(city_line)
site_map.add_child(railway_line)
site_map.add_child(highway_line)

# Display the updated map
site_map


Findings : Launch sites are usually located far away from cities but in very close proximities to coast line and railway.
Highways are moderately close to the launch sites


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
